#### This DL model is CNN based extract and recognize face trained on a single person photos dataset using Generator with Accuracy of 90%

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import os
from keras.preprocessing.image import ImageDataGenerator
# ImageDataGenerator  -> increase size of dataset by genrating diff form img from modifying one img 
from keras.utils import to_categorical,load_img
# load_img -> loading img in grayscale from 3 channel RGB  fromat , it loads img in PIL(Python Imaging Lib) format which procide pyton interpreter access to img proccesing function
# to_categorical -> categorise it , using one-hot encoding

from mtcnn.mtcnn import MTCNN
import keras.utils as image
from PIL import Image

In [72]:
img_width=img_height=128
img_channels=3
img_size=(img_width,img_height)

In [73]:
def extract_face(filename,required_size=(128,128)):
    pixels=plt.imread(filename)
    detector=MTCNN()
    results=detector.detect_faces(pixels)
    print(results)
    if (results!=[]):
        x1,y1,width,height=results[0]['box']
        x2,y2= x1+width , y1+height
        face=pixels[y1:y2,x1:x2]  # extract face
        image=Image.fromarray(face)
        image=image.resize(required_size)
        face_array=np.asarray(image)
        return face_array


In [74]:
try:
    filenames=os.listdir("faces/dp/")
    categories=[]
    file_name=[]
    for f_name in filenames:
        category=f_name.split(".")[0]  
        if (category.isdigit()):
            file_name.append(extract_face("faces/dp/"+f_name))
            categories.append(1)
        else:
            file_name.append(extract_face("faces/dp/"+f_name))
            categories.append(0)

    df=pd.DataFrame({
        "filename":file_name,
        "category":categories
    })
except:
    pass
df.head()


1/1 [==============================] - 0s 138ms/step
[{'box': [246, 77, 165, 214], 'confidence': 0.998663067817688, 'keypoints': {'left_eye': (300, 156), 'right_eye': (380, 158), 'nose': (345, 201), 'mouth_left': (312, 244), 'mouth_right': (366, 243)}}]
1/1 [==============================] - 0s 127ms/step
[{'box': [192, 183, 302, 387], 'confidence': 0.9994335770606995, 'keypoints': {'left_eye': (268, 331), 'right_eye': (413, 342), 'nose': (327, 404), 'mouth_left': (269, 473), 'mouth_right': (392, 480)}}]
1/1 [==============================] - 0s 179ms/step
[{'box': [551, 188, 275, 357], 'confidence': 0.9999978542327881, 'keypoints': {'left_eye': (640, 337), 'right_eye': (773, 341), 'nose': (722, 426), 'mouth_left': (647, 464), 'mouth_right': (756, 464)}}, {'box': [425, 491, 49, 62], 'confidence': 0.7368851900100708, 'keypoints': {'left_eye': (441, 514), 'right_eye': (461, 508), 'nose': (453, 523), 'mouth_left': (448, 539), 'mouth_right': (464, 534)}}]
1/1 [=============================

,filename,category
0,1.jpg,dp
1,10.jpg,dp
2,100.jpg,dp
3,101.jpg,dp
4,102.jpg,dp


In [75]:
df["category"]

0       dp
1       dp
2       dp
3       dp
4       dp
      ... 
199    not
200    not
201    not
202    not
203    not
Name: category, Length: 204, dtype: object

In [76]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,\
     Dropout,Flatten,Dense,Activation,\
     BatchNormalization
model=Sequential()

# relu -> - value to 0 becoz intensity ranges from 0 to 255
# why we get negative value becoz filter has any vakue so fter multiplying we get - value 
# Dropout layer prevent over fitting by randomly setting input to 0 with freq of rate at each step during training time
# varience low & low bais
# batch normalization -> feature scalling 0 to 1 , to reduce range betw ele in matrix like [1,0,111,1,1] 
# Flatten -> 2d to 1d
# Dense -> connected nueral network

# Convolutional Block 1
# using 332 filters in 1st CB , (3,3) -> size of feature detector
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(img_width,img_height,img_channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))  #25% chance of setting the input 0
# Convolutional Block 1
model.add(Conv2D(64,(3,3),activation='relu'))     # no input_shape coz 2nd layer get attach from 1st layer
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
# Convolutional Block 1
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

In [77]:
model.add(Flatten())
# 1st hidden layer
model.add(Dense(512,activation='relu'))    
model.add(BatchNormalization())
model.add(Dropout(0.5))
# 2nd hidden layer
model.add(Dense(2,activation='softmax'))

model.compile(loss='categorical_crossentropy',
            optimizer='rmsprop',
            metrics=['accuracy'])

In [78]:
# Callbacks - Actions perofrmed on various stage of training, atart and end of epochs,before or after single batch
# Do early stopping , periodically saving model, get a view on internal states and stats of a model training
# EARLY STOPPING -> Stop training when metric has stopped improving like if i have 100 epoch then if accu is not increasing till 10(patience) through out the epoch it will stop training
# It reduce the LR when matrics has stop improving 
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
callbacks = [earlystop,learning_rate_reduction]

In [79]:
df["category"] = df["category"].replace({0:'not',1:'dp'})
train_df,validate_df = train_test_split(df,test_size=0.20,
  random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)
total_train=train_df.shape[0]
total_validate=validate_df.shape[0]

In [80]:
batch_size=15 # per img generate 15 img
train_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                )
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                 "./faces/dp/",x_col='filename',y_col='category',
                                                 target_size=img_size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size)
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
                                                validate_df, 
                                                "./faces/dp/", 
                                                x_col='filename',
                                                y_col='category',
                                                target_size=img_size,
                                                class_mode=None,
                                                batch_size=batch_size
                                                )
print("test gen")
# error - no photos found issme padho hi mat 
test_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1)
test_generator = train_datagen.flow_from_dataframe(train_df,
                                                 "./faces/dp/",x_col='filename',y_col='category',
                                                 target_size=img_size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size)

Found 162 validated image filenames belonging to 2 classes.
Found 40 validated image filenames.
test gen
Found 162 validated image filenames belonging to 2 classes.


C:\Users\Janak\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\preprocessing\image.py:989: UserWarning: Found 1 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn('Found {} invalid image filename(s) in x_col="{}". '
C:\Users\Janak\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\preprocessing\image.py:989: UserWarning: Found 1 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn('Found {} invalid image filename(s) in x_col="{}". '


In [81]:
epochs=10
history = model.fit_generator(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

C:\Users\Janak\AppData\Local\Temp\ipykernel_10164\1584285789.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10
10/10 [==============================] - 7s 605ms/step - loss: 1.7964 - accuracy: 0.5986 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/10
10/10 [==============================] - 5s 546ms/step - loss: 1.0606 - accuracy: 0.6400 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 3/10
10/10 [==============================] - 5s 537ms/step - loss: 0.7710 - accuracy: 0.6939 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 4/10
10/10 [==============================] - 5s 504ms/step - loss: 0.6418 - accuracy: 0.7347 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 5/10
10/10 [==============================] - 5s 510ms/step - loss: 0.5387 - accuracy: 0.7823 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 6/10
10/10 [==============================] - 5s 513ms/step - loss: 0.4042 - accuracy: 0.8299 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 7/10
10/10 [====

In [82]:
model.save("model1_dp_10epoch.h5")


In [83]:
predict=model.predict_generator(validation_generator,len(validation_generator.filenames))

C:\Users\Janak\AppData\Local\Temp\ipykernel_10164\3792076396.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predict=model.predict_generator(validation_generator,len(validation_generator.filenames))


In [84]:
predict[0]
predict_lable=[np.argmax(i) for i in predict]

In [85]:

predict_lable

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [86]:
predict[0] # dp , other

array([1.0000000e+00, 2.0108061e-08], dtype=float32)

In [90]:
validation_generator[0]

array([[[[0.42352945, 0.36862746, 0.3647059 ],
         [0.427451  , 0.36862746, 0.39607847],
         [0.41960788, 0.36862746, 0.40784317],
         ...,
         [1.        , 1.        , 0.9921569 ],
         [0.9960785 , 1.        , 0.97647065],
         [1.        , 0.98823535, 0.95294124]],

        [[0.41176474, 0.37647063, 0.38823533],
         [0.41960788, 0.37254903, 0.41176474],
         [0.42352945, 0.37254903, 0.40784317],
         ...,
         [1.        , 1.        , 0.9921569 ],
         [0.9960785 , 1.        , 0.9686275 ],
         [1.        , 0.9843138 , 0.9490197 ]],

        [[0.41176474, 0.37647063, 0.39607847],
         [0.42352945, 0.37647063, 0.38431376],
         [0.41960788, 0.37647063, 0.40000004],
         ...,
         [1.        , 1.        , 0.9921569 ],
         [0.9960785 , 1.        , 0.9607844 ],
         [1.        , 0.9803922 , 0.9568628 ]],

        ...,

        [[0.5529412 , 0.48235297, 0.5294118 ],
         [0.6313726 , 0.627451  , 0.6509804 ]